#### **Dans ce notebook, nous allons charger, nettoyer et préparer les données extraites d'Avito. Le résultat final sera enregistré dans un fichier CSV propre prêt pour l'analyse.**

In [12]:
# Les importations nécessaires
import pandas as pd
import numpy as np
import os

On charge le fichier brut


In [13]:
# Chargement du fichier source (sans le modifier)
df = pd.read_csv(
    '../data/raw/avito_location.csv',
    engine='python',
    on_bad_lines='skip',
    na_values=['null']
)

print("Taille initiale :", len(df))
df.head()

Taille initiale : 19682


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce,date_annonce
0,57160527.0,Casablanca,20000 DH,48 m2,Maarif,Appartement,5,0,https://www.avito.ma/fr/maarif/appartements/__...,il y a 6 heures
1,57454598.0,Tanger,6500 DH,90 m2,Castilla,Appartement,4,0,https://www.avito.ma/fr/castilla/appartements/...,il y a 12 heures
2,57465041.0,Casablanca,3800 DH,167 m2,Maarif,NaN,0,0,https://www.avito.ma/fr/maarif/local/G%C3%A9re...,il y a 10 heures
3,57274506.0,Casablanca,17000 DH,155 m2,Racine,Appartement,3,0,https://www.avito.ma/fr/racine/appartements/CM...,il y a 18 minutes
4,57472696.0,Casablanca,2300 DH,56 m2,Oulfa,Appartement,3,0,https://www.avito.ma/fr/oulfa/appartements/App...,il y a 16 minutes


## Nettoyage de base


Dans cette phase, nous supprimons les colonnes inutiles, les doublons et les lignes où les informations cruciales (comme la ville) sont manquantes.

In [14]:
# 1. Supprimer la colonne date_annonce
if 'date_annonce' in df.columns:
    df.drop(columns=['date_annonce'], inplace=True)

# 2. Supprimer les doublons
df.drop_duplicates(inplace=True)

# 3. Supprimer les lignes où ville est null
df.dropna(subset=['ville'], inplace=True)

# 4. Supprimer les lignes où nb_chambres est 0 (ou vide)
df = df[df['nb_chambres'] > 0]

print("Taille après nettoyage de base :", len(df))

Taille après nettoyage de base : 13569


### Prétraitement : Suppression des unités (DH, m2) et conversion en Float


In [15]:
def clean_numeric_col(series, unit):
    return series.astype(str).str.replace(unit, '', case=False)\
                 .str.replace(r'\s+', '', regex=True)\
                 .replace(['nan', 'None', ''], '0')\
                 .astype(float)

# Nettoyer prix (supprimer DH)
df['prix'] = clean_numeric_col(df['prix'], 'DH')

# Nettoyer surface (supprimer m2)
df['surface'] = clean_numeric_col(df['surface'], 'm2')

print("Types après conversion :")
print(df[['prix', 'surface']].dtypes)

Types après conversion :
prix       float64
surface    float64
dtype: object


### Test de la logique d'imputation (Type de bien)

In [16]:
# 1. Trouver l'index de la première ligne où type_bien est null
try:
    idx_test = df[df['type_bien'].isna()].index[0]
    
    print(f"--- AVANT (Index: {idx_test}) ---")
    display(df.loc[[idx_test]])

    # 2. Appliquer la logique de nettoyage et d'imputation
    df['type_bien'] = df['type_bien'].str.strip().str.capitalize()
    
    def impute_type(row):
        if pd.isna(row['type_bien']) or row['type_bien'] == 'None':
            return 'Maison' if row['nb_chambres'] <= 6 else 'Villa'
        return row['type_bien']

    df['type_bien'] = df.apply(impute_type, axis=1)

    print(f"--- APRÈS (Index: {idx_test}) ---")
    display(df.loc[[idx_test]])

except IndexError:
    print("Aucune ligne avec type_bien = null n'a été trouvée dans votre dataset.")

--- AVANT (Index: 35) ---


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce
35,57452829.0,Casablanca,11000.0,84.0,Casablanca Finance City,NaN,3,0,https://www.avito.ma/fr/casablanca_finance_cit...


--- APRÈS (Index: 35) ---


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bains,url_annonce
35,57452829.0,Casablanca,11000.0,84.0,Casablanca Finance City,Maison,3,0,https://www.avito.ma/fr/casablanca_finance_cit...


### Imputation du nombre de salles de bains
**Règle appliquée :**
- Si la valeur est `0` ou `Null` :
    - Si `nb_chambres` ≤ 3  ➔  **1** salle de bain.
    - Sinon  ➔  **(nb_chambres / 2) - 1**.

In [17]:
def fix_bathrooms(row):
    # Si c'est 0 ou NaN
    if row['nb_salle_de_bains'] == 0 or pd.isna(row['nb_salle_de_bains']):
        if row['nb_chambres'] <= 3:
            return 1
        else:
            return (row['nb_chambres'] / 2) - 1
    return row['nb_salle_de_bains']

df['nb_salle_de_bains'] = df.apply(fix_bathrooms, axis=1)

### Traitement des Prix et Surfaces à 0
**Stratégie d'imputation :**
1. **Villes cibles** (Casablanca, Rabat, Tanger, Marrakech) : Remplacement des `0` par la moyenne groupée par *Ville* et *Type de bien*.
2. **Fallback global** : Pour les surfaces restantes à `0`, application de la moyenne générale par *Type de bien*.

In [18]:
target_cities = ['Casablanca', 'Rabat', 'Tanger', 'Marrakech']

for col in ['prix', 'surface']:
    # 1. On calcule la table de référence des moyennes par Ville et par Type
    # On ne prend que les valeurs strictement supérieures à 0
    df_valid = df[df[col] > 0]
    lookup_villes = df_valid.groupby(['ville', 'type_bien'])[col].mean()
    
    # 2. On calcule aussi une moyenne globale par Type de bien (au cas où une ville n'a que des 0)
    lookup_global = df_valid.groupby('type_bien')[col].mean()

    # 3. Création du masque pour les lignes à 0 dans les villes cibles
    mask = (df[col] == 0) & (df['ville'].isin(target_cities))

    # 4. Remplacement intelligent
    # On essaie d'abord la moyenne Ville + Type
    df.loc[mask, col] = df.loc[mask].apply(
        lambda row: lookup_villes.get((row['ville'], row['type_bien']), 
                    lookup_global.get(row['type_bien'], 0)), 
        axis=1
    )

    # 5. Sécurité finale : s'il reste des 0 ou des NaN (pour les villes hors liste)
    df[col] = df[col].replace(0, np.nan) # On transforme les 0 en NaN pour utiliser fillna
    df[col] = df[col].fillna(df.groupby('type_bien')[col].transform('mean'))
    
    # Si vraiment il n'y a aucune donnée pour un type de bien (ex: seul terrain du site)
    df[col] = df[col].fillna(0)

print("Nettoyage des prix et surfaces terminé sans NaN.")

Nettoyage des prix et surfaces terminé sans NaN.


### Formatage final et conversion des types de données

In [19]:
# Nettoyage spécifique de l'ID pour éviter le ".0"
df['id'] = pd.to_numeric(df['id'], errors='coerce').fillna(0).astype(int).astype("string")

# Conversion finale des types (pour avoir bien le type 'string' et non 'object')
df = df.astype({
    'ville': 'string',
    'quartier': 'string',
    'type_bien': 'string',
    'url_annonce': 'string',
    'nb_chambres': 'int',
    'nb_salle_de_bains': 'int',
    'prix': 'float',
    'surface': 'float'
})

print("Vérification des types :")
print(df.dtypes)

Vérification des types :
id                   string[python]
ville                string[python]
prix                        float64
surface                     float64
quartier             string[python]
type_bien            string[python]
nb_chambres                   int64
nb_salle_de_bains             int64
url_annonce          string[python]
dtype: object


### Nettoyage, Filtrage et Imputation des types de biens

In [20]:
# 1. Nettoyage des espaces blancs et uniformisation de la casse (Première lettre en Majuscule)
df['type_bien'] = df['type_bien'].str.strip().str.capitalize()
df['ville'] = df['ville'].str.strip().str.capitalize()

# 2. Supprimer les types indésirables avec certitude
excluded_types = ['Bureau', 'Magasin', 'Terrain',"Commerce"]
df = df[~df['type_bien'].isin(excluded_types)]

# 3. Imputer type_bien si null (règle nbr_chambres)
def impute_type(row):
    if pd.isna(row['type_bien']) or row['type_bien'] == 'None':
        return 'Maison' if row['nb_chambres'] <= 6 else 'Villa'
    return row['type_bien']

df['type_bien'] = df.apply(impute_type, axis=1)

print("Types de biens restants :", df['type_bien'].unique())

Types de biens restants : ['Appartement' 'Villa' 'Maison']


### Contrôle Qualité 

In [21]:
print("=== AUDIT DE QUALITÉ DES DONNÉES FINALES ===")

# 1. Analyse des colonnes critiques (Types, NaN et Zéros)
critique_cols = ['prix', 'surface', 'nb_chambres', 'nb_salle_de_bains']
audit = pd.DataFrame({
    'Type': df.dtypes,
    'Manquants (NaN)': df.isna().sum(),
    'Valeurs à 0': (df == 0).sum()
})

# 2. Affichage du tableau de bord
display(audit)

# 3. Vérifications de logique métier (Alertes rapides)
nan_total = df.isna().sum().sum()
zero_total = (df[critique_cols] == 0).sum().sum()
types_interdits = df['type_bien'].str.lower().isin(['terrain', 'bureau', 'magasin', 'commerce']).sum()

print("-" * 40)
print(f"✅ STATUT GLOBAL : {'PROPRE' if (nan_total + zero_total + types_interdits) == 0 else 'À RÉVISER'}")
print(f"📍 Lignes totales : {len(df)}")
print(f"🔸 Valeurs NaN    : {nan_total}")
print(f"🔸 Valeurs à 0    : {zero_total} (Colonnes critiques)")
print(f"🔸 Types exclus   : {types_interdits} restants")
print(f"🔸 Quartiers vides: {df['quartier'].isna().sum()}")
print("-" * 40)

# 4. Aperçu rapide de la distribution
print("\n🏠 RÉPARTITION DES BIENS :")
print(df['type_bien'].value_counts())

=== AUDIT DE QUALITÉ DES DONNÉES FINALES ===


,Type,Manquants (NaN),Valeurs à 0
id,string[python],0,0
ville,string[python],0,0
prix,float64,0,0
surface,float64,0,0
quartier,string[python],0,0
type_bien,object,0,0
nb_chambres,int64,0,0
nb_salle_de_bains,int64,0,0
url_annonce,string[python],0,0


----------------------------------------
✅ STATUT GLOBAL : PROPRE
📍 Lignes totales : 13516
🔸 Valeurs NaN    : 0
🔸 Valeurs à 0    : 0 (Colonnes critiques)
🔸 Types exclus   : 0 restants
🔸 Quartiers vides: 0
----------------------------------------

🏠 RÉPARTITION DES BIENS :
type_bien
Appartement    10483
Maison          1976
Villa           1057
Name: count, dtype: int64


In [22]:
df = df.rename(columns={"nb_salle_de_bains": "nb_salle_de_bain"})

### Sauvegarde des données nettoyées (Export Final)

In [23]:
# 1. Définition du chemin de sortie
output_dir = "../data/clean_data"
output_path = os.path.join(output_dir, "avito_location_clean.csv")

# 2. Création du dossier s'il n'existe pas
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 3. Sauvegarde du DataFrame
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Félicitations ! Le fichier a été sauvegardé avec succès.")
print(f"📍 Emplacement : {output_path}")
print(f"Nombre de lignes finales : {len(df)}")


✅ Félicitations ! Le fichier a été sauvegardé avec succès.
📍 Emplacement : ../data/clean_data\avito_location_clean.csv
Nombre de lignes finales : 13516
